#### Clustering Operation exposed as a Service

This file does the following:
* Creates a model with k-means clustering
* Predicts the label and returns it

In [3]:
#pip install bottle

In [4]:
# imports
from bottle import route, run     

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
%matplotlib inline
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle

In [6]:
kmeans = None
labels = None

Method to load the csv file and perform k-means clustering

In [8]:
def perform_clustering(path=r"/Users/bolunsun/Downloads/html_agent_ui/src/cleanup_url\features.csv", n_clusters=2, sep='|'):
    """
    Loads the csv file passed in the path argument and perform k-means clustering
    By default - columns named X1 and X2 will be used as features
    """
    df = pd.read_csv(path, sep=sep)
    X = np.array(df[['X1','X2']])
    
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X)
    
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    #colors = ["g.","r.","c.","y.","b."]

    #for i in range(len(X)):
        #print("coordinate:",X[i], "label:", labels[i])
        #plt.plot(X[i][0], X[i][1], colors[labels[i]], markersize = 10)


    #plt.scatter(centroids[:, 0],centroids[:, 1], marker = "x", s=150, linewidths = 5, zorder = 10)
    
    # dump the model
    pickle.dump(kmeans, open('temp.pkl', 'wb')) 

Method to predict the label of the inputs passed

In [10]:
def predict_label(X1=5, X2=0):
    """
    Predict and return the label of the parameters passed.
    Default X1=5, X2=0 (To get most confident label)
    """
    model = pickle.load(open('temp.pkl', 'rb'))
    test_values = np.array([[X1, X2]])
    return model.predict(X=test_values)[0]

#### Web Service Part Start

Get Request to perform k-means and store the model

In [13]:
@route('/hello')
def hello():
    return "Hello World!"

In [14]:
@route('/kmeans')
def invokeKMeans():
    """
    Exposed method to perform clustering invoked using - http://localhost:port/kmeans
    """
    try:
        perform_clustering()
        return '1'
    except:
        return '0'


In [15]:
@route('/predict/<values:re:.*>')
def predict(values):
    """
    Exposed method to get the label of the values passed.
    Values should be passed in the form - http://localhost:port/predict/X1|X2
    """
    if(len(values) > 0):
        values = str(values)
        X1 = values.split('|')[0]
        X2 = values.split('|')[1]
       
        prediction = predict_label(X1=X1, X2=X2)
        
    
    prediction =  predict_label()
    
    return str(prediction)

In [ ]:
run(host='localhost', port=12007, debug=False)

Bottle v0.12.25 server starting up (using WSGIRefServer())...
Listening on http://localhost:12007/
Hit Ctrl-C to quit.

127.0.0.1 - - [05/Sep/2024 00:24:12] "GET /kmeans HTTP/1.1" 200 1
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/bottle.py", line 876, in _handle
    return route.call(**args)
           ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/bottle.py", line 1759, in wrapper
    rv = callback(*a, **ka)
         ^^^^^^^^^^^^^^^^^^
  File "/var/folders/gc/1xbjfy4j4pgfbsqcknqj_jz80000gn/T/ipykernel_76578/430888322.py", line 12, in predict
    prediction = predict_label(X1=X1, X2=X2)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gc/1xbjfy4j4pgfbsqcknqj_jz80000gn/T/ipykernel_76578/1306129270.py", line 6, in predict_label
    model = pickle.load(open('temp.pkl', 'rb'))
                        ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py"